In [71]:
from googleapiclient.discovery import build

import pandas as pd
from IPython.display import JSON

In [72]:
api_key = ''

In [73]:
channel_ids= ['', # Siattle data guy channel
                # add more channels here
                ]

In [74]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(api_service_name, api_version, developerKey=api_key)

* root
  * ...
  *  items     --->  *loop to dataframe*

     *  snippet
          *  title
          *  description
          *  custom URL
          *  publised at
          *  ...
     * content details
          * related playlists
            * likes
            * upload
     * statistics
          * view count
          * subscriber count
          * hidden subscriber count
          * video count
  

In [75]:
type(response)
response

{'kind': 'youtube#playlistListResponse',
 'etag': 'BVNt3_f92l_BFhtQ5ipFuVteIDM',
 'nextPageToken': 'CAUQAA',
 'pageInfo': {'totalResults': 11, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#playlist',
   'etag': 'Y5gSMD_49aO4XazSgegfxnaOcGU',
   'id': 'PLXRKPZRrlvE5P7-qWpUrQ8mqsfZD_Mulw',
   'snippet': {'publishedAt': '2022-03-29T14:14:08Z',
    'channelId': 'UCmLGJ3VYBcfRaWbP6JLJcpA',
    'title': 'Seattle Data Guy Lives',
    'description': '',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/M6mRSsLI19I/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/M6mRSsLI19I/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/M6mRSsLI19I/hqdefault.jpg',
      'width': 480,
      'height': 360}},
    'channelTitle': 'Seattle Data Guy',
    'localized': {'title': 'Seattle Data Guy Lives', 'description': ''}},
   'contentDetails': {'itemCount': 3}},
  {'kind': 'youtube#playlist'

### Get channel statistics

In [76]:
def get_channel_stats(youtube, channel_ids):

    all_data = []
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=",".join(channel_ids)
    )
    response = request.execute()
    #JSON(response)
    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'view': item['statistics']['viewCount'],
                'totalView': item['statistics']['videoCount'],
                'playListId': item['contentDetails']['relatedPlaylists']['uploads']
                }
        all_data.append(data)
    return pd.DataFrame(all_data)

In [77]:
get_channel_stats(youtube, channel_ids)

,channelName,subscribers,view,totalView,playListId
0,Seattle Data Guy,25900,1196319,122,UUmLGJ3VYBcfRaWbP6JLJcpA


### Get video ids

In [94]:
playlist_id = "UUmLGJ3VYBcfRaWbP6JLJcpA"

def get_video_ids(youtube, playlist_id):

    video_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet, contentDetails",
        playlistId=playlist_id,
        maxResults = 50, # youtube data API documentation /playlistitems / list
    )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="snippet, contentDetails",
            playlistId=playlist_id,
            maxResults = 50, # youtube data API documentation /playlistitems / list
        )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')

    return video_ids

In [96]:
video_ids = get_video_ids(youtube, playlist_id)

ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

In [95]:
video_ids

NameError: name 'video_ids' is not defined